# Notebook 1.

In this notebook we are going to have a look at different types of machine learning in action. 
We will start by looking at some **unsupervised learning** algorithms, and then move on to **supervised learning**.

---

In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

## 1. Unsupervised Learning
<!-- requirement: images/rbm_graph.png -->
<!-- requirement: images/map_compression.png -->
<!-- requirement: images/nmf_example.png -->


Unsupervised learning is a type of learning that occurs when you only have features $X$ but not labels $y$. In this section, we will discuss **clustering**, an unsupervised learning technique which finds groups of similar data points.

Real-life situations where we might want to group data points into several clusters include:
* Group customers into segments according to their behaviour. 
* Group genes and proteins that have similar functionality.
* Group stocks with similar price fluctuations.
* Identifying football players who play the similar roles to inform signing decisions.

Can you think of any more?

Clustering is ***not*** classification.  There is no "right" answer.  Instead, there are a number of algorithms to use and metrics to judge them by.  Below, we generate a cloud of points and use an algorithm called k-means to cluster them into three groups.  (Note that this is an input.  How do we choose the right number?)

The first thing we will do is create an example data set. 

In [ ]:
# generate a blob of points
import numpy as np
from matplotlib import pylab as plt

np.random.seed(42)
X1 = np.vstack((np.random.normal(0, 0.5, (100, 2)) + np.array([0, 1]),
                np.random.normal(0, 0.5, (100, 2)) + np.array([-0.5, -0.7]),
                np.random.normal(0, 0.5, (100, 2)) + np.array([0.5, -0.7])))

# plot those points
plt.scatter([el[0] for el in X1], [el[1] for el in X1])

**Note**: in real-world situations, our data will very likely have more than 2 dimensions. But for the purposes of this demonstration, let's just consider this two dimensional dataset. 

Ok, so now we have our data, the next step is to apply a clustering algorithm to find data points which are similar and might belong to the same group. 

To visualise what is happening at different stages in the clustering algorithm, let's build a function which incorporates sklean's KMeans algorithm along with the matplotlib library.

In [ ]:
from sklearn.cluster import KMeans

def plot_kmeans(X, n, start):
    def func(step=0):
        iters = step // 2
        if iters:
            km = KMeans(n_clusters=n, max_iter=iters, n_init=1, init=start)
            km.fit(X)
            centers = km.cluster_centers_.T
        else:
            centers = start.T
        if step % 2:
            km = KMeans(n_clusters=n, max_iter=iters+1, n_init=1, init=start)
            km.fit(X)
        plt.scatter(*X.T, c=(km.labels_ if step else '0.5'), cmap=plt.cm.plasma)
        plt.scatter(*centers, c=range(n), cmap=plt.cm.plasma, marker='*', s=100,
                    linewidth=1, edgecolors=['k' if i else '0.9' for i in range(n)])
        plt.title(['Set centroids', 'Assign clusters'][step % 2])
    return func

Now that we have this function, let's test it out on our data. 

Run the following cell with k=3. Remember what the clusters look like after 10 iterations, then run the cell again. Do you notice anything?

In [ ]:
# set k - the number of clusters
k=3 

# Plot our data with the initial starting centroids, and then again after 10 iterations of the k-means algorithm. 
x=np.random.randint(0,100-k)
plt.subplot(121)
plot_kmeans(X1, k, X1[x:x+k])(0)
plt.axis('image')
plt.title('Points')
plt.subplot(122)
plot_kmeans(X1, k, X1[x:x+k])(20)
plt.axis('image')
plt.title('Clusters after 10 iterations');

## K-Means clustering

We saw K-Means in action above. Here are the mathematical details.

The specification of $K$-means is simple: assign a collection of clusters $C_k$ that minimize

$$ \mbox{argmin}_C \sum_{k=1}^K \sum_{X_j \in C_k} \| X_{j\cdot} - \mu_k \|_2^2 $$

where $\mu_k$ is the center of the points in $C_k$.  The algorithm to implement this is simple:

Initialize $\mu_k$ (with possibly random values).  Then iterate between
1. Assign $X_{j\cdot}$ to the cluster $C_k$ that minimizes $\|X_{j\cdot} - \mu_k\|_2^2$.
1. Recompute $\mu_k$ by averaging over all the points $X_{j\cdot}$ in the cluster $C_k$.

Notice that both iterative steps lower the objective (the algorithm is greedy) and there are only a finite number of possible partitions of the points $X_{j\cdot}$ so the algorithm is guaranteed to converge.  The converged solution may not be globally optimal.

In [ ]:
from ipywidgets import interact

# Set your value for k
k=3
x=np.random.randint(0,100-k)

interact(plot_kmeans(X1, k, X1[x:x+k]), step=(0, 21, 1));

**Questions**:
1. For $K$-Means, what happens if the features have very different scales?
2. How do you pick the best value of k?

---

**Notes on K-Means**:
* It is sensitive to outliers.
* Clusters depend on initialisation.
* It has difficulties with non-spherical clusters and clusters of different sizes.

## Metrics for clustering


For unsupervised learning, we will assume a set of feature vectors $x_j$.  These correspond to the rows of the features matrix $X_{ji}$.  These algorithms revolve around clustering, i.e. assigning each row $X_{j \cdot}$ to a cluster $C_k$ such that the rows that share a cluster are more *similar* than ones from different clusters.  There are several ways to do this, some which require ground truth labeling and some which do not.  

Below we present mathematical details for the **Silhouette Coefficient**, a common metric used in clustering.
The Silhouette Coefficient, measures how well-defined your clusters are (dense and well-separated clusters = good).


If $a_j$ is the mean distance between a point $X_{j \cdot}$ and the other points in the same cluster $C_k$ , and $b_j$ is the mean distance between $X_{j \cdot}$ and the other points in the next nearest cluster $C_k'$, then the coefficient is given by
$$ \frac{b-a}{\max(a, b)}\,. $$


There are a plethora of metrics.  For more information, including advantages and disadvantages of each metric, see this [Scikit Learn page on Clustering Metrics](http://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation).

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn import datasets

# Fit KMeans onto the Iris dataset
dataset = datasets.load_iris()
X = dataset.data
k_means = KMeans().fit(X)
y = k_means.predict(X)

# Compute Silhoutte score on clusters:
print("Silhouette Score", metrics.silhouette_score(X, y, metric='euclidean'))


---

We won't touch on dimensionality reduction here, but there's a really great brief article on PCA (principle component analysis) on [Stats Stack Exchange](https://stats.stackexchange.com/questions/10251/how-to-find-principal-components-without-matrix-algebra) if you're interested.

---

## 2. Supervised Learning

In this section we will cover regression and regression metrics, using Linear Regression as an example. You should be familiar with this from the example in the slides.

**Regression** is the general term for supervised learning problems where we try to predict the value of a continuous variable.  That is, we want to build a model $f$ that approximates the relationship between features $X$ and labels $y$ so that 

$$ f(X_j) \approx y_j $$

for each observation $(X_j,y_j)$.  Here the $y_j$ are real numbers, because $y$ is continuous.

## Linear Regression

In a linear regression model, $f$ is simply a linear function of the features.  If $X_{ji}$ is the $i$th feature of observation $X_j$, then

$$ f(X_j) = \sum_{i=1}^p X_{ji}\beta_i + \beta_0 $$

where the coefficients $\beta_i$ and intercept term $\beta_0$ are values learned by the model during training, and $p$ is the number of features.  This is sometimes written more compactly as

$$ f(X) = X \cdot \beta + \beta_0 $$

where $X=(X_{ji})$ is a matrix, $\beta = (\beta_i)$ is a column vector, and $\beta_0$ is added elementwise to the result of the product. 

Let's look at a simple example.  First we'll generate random data with a single feature.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(0) # for consistency

X = np.linspace(-10,10,30)
y = 2 * X + 3 + 4*np.random.randn(X.shape[0])
plt.plot(X, y, 'o');

Training a linear model on this data means finding the coefficient $\beta$ and intercept $\beta_0$ so that $f(X) = X\cdot\beta+\beta_0$ best approximates our label variable $y$.  This is equivalent to finding the line $y=mx+b$ that best fits the plotted points.

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X.reshape(-1,1), y) # reshape to column vector
print('Intercept = {:.2f}'.format(lr.intercept_))
print('Slope = {:.2f}'.format(lr.coef_[0]))

y_pred = lr.predict(X.reshape(-1,1))

plt.plot(X, y, 'o', label='training data')
plt.plot(X, y_pred, label='model prediction')
plt.legend();

## Regression Metrics

In the preceding discussion, we used phrases like "best approximates" and "best fits".  In order to actually do machine learning, we need to be more concrete.  That is, we need to define metrics that quantify how well a model fits a given set of data.  Metrics provide a cost function to minimize during training and they act as a benchmark to evaluate trained models.

**Mean Squared Error** is the usual metric:

$$ \frac{1}{n} \sum_j \left[f(X_{j}) - y_j\right]^2. $$

Unfortunately, this is susceptible to outliers. When this is an issue, **Mean Absolute Error** can be better:

$$ \frac{1}{n} \sum_j \left|f(X_{j}) - y_j\right|. $$

You've probably heard of **$R^2$** or the **Coefficient of Determination**. Although it's usually defined in a linear regression context, it's actually a very general idea: it measures the fraction of the error explained by the model $f$ versus the fraction of the error explained by a naive model that assumes the mean value of $y$ (i.e. the variance of $y$):

$$ 1 - \dfrac{\sum_j \left[f(X_{j}) - y_j\right]^2}{\sum_j \left(\overline y - y_j\right)^2} \qquad \mbox{where} \qquad \overline y = \frac{1}{n}\sum_j y_j \,.$$

**Questions:**

1. For a list of scalar values $z_1,\ldots,z_n$, the **mean** $\overline z$ is the quantity that minimizes the squared error:
 $$ \frac{d}{dz} \sum_j \left|z - z_j\right|^2 = 0$$
 $$ \frac{d}{dz} \left(Nz^2 - 2z(z_1 + z_2 + ...) + z_1^2 + z_2^2 + ...\right) = 0$$
 $$ 2Nz - 2(z_1 + z_2 + z_3 + ...) = 0$$

 $$ z = \frac{z_1 + z_2 + z_3 + ...}{N} = \overline z$$

 Do you know what quantity comes from minimizing the absolute error?
 $$ \mbox{argmin}_z \sum_j \left|z - z_j\right| $$
 Does this help explain why Absolute Error is less susceptible to outliers?
1. How does each of these metrics scale as you scale the labels ($y$'s) in our data set?

Note that Mean Absolute Error (MAE) and Mean Squared Error (MSE) are both difficult to interpret without context because they depend on the scale of the data.  However, because $R^2$ has a fixed range, a value close to 1 always means that the model is fitting the data fairly well.

## Adding Features

One apparent limitation of linear models is that they cannot capture non-linear behavior.  For example, suppose we try to fit a linear model to data with a quadratic shape:


In [ ]:
X = np.linspace(-10,10,30)
y = -0.8 * X * X + 2 * X + 3 + 8*np.random.randn(X.shape[0])

lr = LinearRegression()
lr.fit(X.reshape(-1,1), y) # reshape to column vector

y_pred = lr.predict(X.reshape(-1,1))

print("R^2:", metrics.r2_score(y, y_pred))

plt.plot(X, y, 'o')
plt.plot(X, y_pred);

As expected, the model fits the data poorly.  However, if we have a vague idea of what sort of pattern the data follows, then we can help our model by creating artificial features that encode the corresponding nonlinearities.  In this case, it seems that the shape of the data is quadratic, so we can add a feature whose value is always equal to $X^2$.

In [ ]:
X_quad = np.vstack((X,X**2)).T # Augmented feature matrix with columns X, X**2

lr.fit(X_quad, y)

y_pred = lr.predict(X_quad)

print("R^2:", metrics.r2_score(y, y_pred))

plt.plot(X, y, 'o')
plt.plot(X, y_pred)